In [ ]:
import cv2
import pytesseract
import numpy as np
import openai

# Set your OpenAI API key
openai.api_key = 'YOUR_OPENAI_API_KEY'

# Step 1: Function to extract text from an image
def extract_text_from_image(image_path):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    text = pytesseract.image_to_string(gray)
    return text

# Step 2: Function to preprocess the image for CLIP
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = img.astype('float32') / 255.0
    return img

# Step 3: Function to preprocess the text for prediction
def preprocess_text(text):
    # Dummy function as we are using CLIP for text classification
    return text

# Step 4: Function to perform image classification using CLIP
def classify_image(image_path, text):
    img = preprocess_image(image_path)
    inputs = {
        'images': [img],
        'text': text
    }
    result = openai.Classification.create(model="clip-v1", inputs=inputs)
    label = result['choices'][0]['label']
    return label

# Step 5: Load the Nature Model for Nature Classification
nature_model = keras.models.load_model('nature_model.h5')

# Step 6: Function to preprocess the image for Nature Classification
def preprocess_image_for_nature(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = img.astype('float32') / 255.0
    return np.expand_dims(img, axis=0)

# Step 7: Function to perform nature classification using the loaded model
def classify_nature(image_path):
    img = preprocess_image_for_nature(image_path)
    nature_class_probabilities = nature_model.predict(img)
    nature_class_label = np.argmax(nature_class_probabilities, axis=-1)[0]
    return nature_class_label

# Step 8: Load the TfidfVectorizer used for text features extraction (from previous training)
vectorizer = joblib.load('vectorizer.pkl')

# Step 9: Function to extract text features using the loaded vectorizer
def extract_text_features_from_image(image_path):
    text = extract_text_from_image(image_path)
    text_features = vectorizer.transform([text])
    return text_features

# Step 10: Function to perform text classification using CLIP
def classify_text(image_path):
    text_features = extract_text_features_from_image(image_path)
    text_features = text_features.toarray()
    text = extract_text_from_image(image_path)
    text_class_label = classify_image(image_path, text)
    return text_class_label

# Example usage
image_path = 'image.jpg'
predicted_text_class = classify_text(image_path)
predicted_nature_class = classify_nature(image_path)

print("Predicted Text Class Label:", predicted_text_class)
print("Predicted Nature Class Label:", predicted_nature_class)